In [10]:
pip install requests beautifulsoup4


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

URL = "https://www.huila.gov.co/cultura-y-turismo/publicaciones/6130/patrimonio-tangible-inmueble/"

def norm_txt(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return " ".join(s.split()).strip().lower()

resp = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})
resp.encoding = "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

tables = pd.read_html(resp.text)

registros = []
for df in tables:
    categoria_actual = None
    for r in df.itertuples(index=False):
        row_vals = [str(x) if pd.notna(x) else "" for x in r]
        row_norm = [norm_txt(v) for v in row_vals]
        row_join = " ".join(row_norm)

        # Detectar fila con categoría
        if any("categoria" in v for v in row_norm):
            for val in row_vals:
                if val.strip() != "":
                    categoria_actual = val.strip()
                    break
            continue

        # Saltar encabezados
        if ("municipio" in row_join and "denominacion" in row_join) or ("municipio" in row_join and "decreto" in row_join):
            continue

        # Filas con 3 valores válidos
        valores_utiles = [v.strip() for v in row_vals if v.strip() != ""]
        if len(valores_utiles) == 3:
            municipio, denominacion, decreto = valores_utiles
            registros.append({
                "Categoria": categoria_actual,
                "Municipio": municipio,
                "Denominacion": denominacion,
                "Decreto": decreto
            })

df_final = pd.DataFrame(registros)
df_final.to_csv("patrimonio_huila.csv", index=False, encoding="utf-8")
print(f"✅ Archivo generado con {len(df_final)} registros")


✅ Archivo generado con 84 registros


/tmp/ipython-input-3102109276.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import time

BASE_URL = "https://www.huila.gov.co"
URL = "https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/"

def norm_txt(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return " ".join(s.split()).strip()

resp = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})
resp.encoding = "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

# --- Extraer lista de museos y sus links ---
museos = []
for a in soup.select("#infoPrincipal a[href]"):
    nombre = norm_txt(a.get_text())
    link = a["href"].strip()
    if link.startswith("http"):
        url = link
    else:
        url = BASE_URL + link
    museos.append({"Nombre": nombre, "URL": url})

# --- Para cada museo, entrar y extraer info ---
detalles = []
for m in museos:
    print(f"⏳ Procesando {m['Nombre']} -> {m['URL']}")
    info = ""
    try:
        r = requests.get(m["URL"], headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        r.encoding = "utf-8"
        s = BeautifulSoup(r.text, "html.parser")

        # Extraer todos los <p> con texto
        parrafos = [norm_txt(p.get_text(" ", strip=True)) for p in s.select("div.modContent p") if p.get_text(strip=True)]
        info = " | ".join(parrafos)

    except Exception as e:
        print(f"⚠️ Error con {m['Nombre']}: {e}")
        # Se conserva el nombre, pero la info queda vacía

    detalles.append({
        "Nombre": m["Nombre"],
        "URL": m["URL"],
        "Info": info
    })
    time.sleep(1)  # pequeña pausa para no saturar el servidor

df_museos = pd.DataFrame(detalles)
df_museos.to_csv("museos_huila.csv", index=False, encoding="utf-8")
print(f"✅ Archivo generado con {len(df_museos)} museos")


⏳ Procesando Inicio -> https://www.huila.gov.co/cultura-y-turismo/
⏳ Procesando Compartir -> https://www.huila.gov.co#
⏳ Procesando Buscar -> https://www.huila.gov.co#
⏳ Procesando Facebook -> https://www.facebook.com/sharer/sharer.php?u=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Twitter -> https://twitter.com/intent/tweet?url=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Linkedin -> https://www.linkedin.com/shareArticle?mini=true&url=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Whatsapp -> https://www.huila.gov.cowhatsapp://send?text=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⚠️ Error con Whatsapp: HTTPSConnectionPool(host='www.huila.gov.cowhatsapp', port=443): Max retries exceeded with url: /send?text=https://www.huila.gov.co/cultura-y-turismo/publicacio

In [13]:
import pandas as pd
import re

# --- Leer el CSV generado ---
df = pd.read_csv("/content/museos_huila.csv", encoding="utf-8")

# --- Eliminar los primeros 16 registros ---
df = df.iloc[16:-2].reset_index(drop=True)

# --- Función para extraer ubicación ---
def extraer_ubicacion(info: str) -> str:
    if not isinstance(info, str) or info.strip() == "":
        return ""

    # Normalizamos texto
    text = info.lower()

    # Buscar expresiones comunes de ubicación
    patrones = [
        r"(municipio de [^\|]+)",   # Ej: Municipio de Neiva
        r"(carrera\s*\d+\s*#\s*\d+\s*-\s*\d+[^|]*)",  # Ej: Carrera 5 # 21 - 81
        r"(calle\s*\d+[^|]*)",     # Ej: Calle 10
        r"(avenida\s*[^\|]+)",     # Ej: Avenida principal
        r"(centro cultural[^\|]*)" # Ej: Centro Cultural y de convenciones...
    ]

    for pat in patrones:
        m = re.search(pat, text)
        if m:
            return m.group(1).strip()

    return ""

# --- Crear nueva columna con la ubicación ---
df["Ubicacion"] = df["Info"].apply(extraer_ubicacion)

# --- Guardar nuevo archivo ---
df.to_csv("museos_huila_limpio.csv", index=False, encoding="utf-8")

print("✅ Archivo generado con ubicaciones extraídas")


✅ Archivo generado con ubicaciones extraídas
